### Machine Learning Lab08 Exercise: CNN-based Image Classifier for CIFAR10 Dataset

#### 1. Import packages and device

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#### 2. Import data

In [15]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./CIFAR10_dataset', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./CIFAR10_dataset', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


#### 3. Design CNN layers

In [7]:
# class Net(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.conv1 = nn.Conv2d(3, 9, 5)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(9, 36, 5)

#         self.fc1 = nn.Linear(36 * 5 * 5, 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 10)

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = torch.flatten(x, 1)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x


class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(
            in_channels=3, out_channels=64, kernel_size=5, stride=1, padding=0
        )
        self.pool = torch.nn.MaxPool2d(kernel_size=3, stride=2)
        self.conv2 = torch.nn.Conv2d(64, 64, 5)
        self.fc1 = torch.nn.Linear(64 * 4 * 4, 384)
        self.fc2 = torch.nn.Linear(384, 192)
        self.fc3 = torch.nn.Linear(192, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net().to(device=device)

#### 4. CNN Training

In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
for epoch in range(16):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 2000 == 1999:
            print(f"[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}")
            running_loss = 0.0

print("Finished Training")

[1,  2000] loss: 0.753
[1,  4000] loss: 0.740
[1,  6000] loss: 0.757
[1,  8000] loss: 0.773
[1, 10000] loss: 0.807
[1, 12000] loss: 0.757
[2,  2000] loss: 0.684
[2,  4000] loss: 0.730
[2,  6000] loss: 0.725
[2,  8000] loss: 0.726
[2, 10000] loss: 0.739
[2, 12000] loss: 0.747
[3,  2000] loss: 0.680
[3,  4000] loss: 0.682
[3,  6000] loss: 0.694
[3,  8000] loss: 0.699
[3, 10000] loss: 0.688
[3, 12000] loss: 0.690
[4,  2000] loss: 0.626
[4,  4000] loss: 0.662
[4,  6000] loss: 0.663
[4,  8000] loss: 0.681
[4, 10000] loss: 0.691
[4, 12000] loss: 0.685
[5,  2000] loss: 0.623
[5,  4000] loss: 0.628
[5,  6000] loss: 0.637
[5,  8000] loss: 0.671
[5, 10000] loss: 0.662
[5, 12000] loss: 0.655
[6,  2000] loss: 0.585
[6,  4000] loss: 0.611
[6,  6000] loss: 0.631
[6,  8000] loss: 0.641
[6, 10000] loss: 0.654
[6, 12000] loss: 0.659
[7,  2000] loss: 0.592
[7,  4000] loss: 0.604
[7,  6000] loss: 0.596
[7,  8000] loss: 0.620
[7, 10000] loss: 0.617
[7, 12000] loss: 0.620
[8,  2000] loss: 0.587
[8,  4000] 

#### 5. Model Evaluation

In [20]:
dataiter = iter(testloader)
images, labels = next(dataiter)
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy of the network on the 10000 test images: {100 * correct // total} %")

Accuracy of the network on the 10000 test images: 68 %


#### 6. Save & Load CNN Model

In [16]:
PATH = './lab08_cifar_net.pth'
torch.save(net.state_dict(), PATH)

In [ ]:
net = Net().to(device)
net.load_state_dict(torch.load(PATH))